# Objects in boxes

## Objective
* We want to put $N$ objects into a row of $N$ boxes.

* Boxes are aligned from left to right (if $i < i'$, box $i$ is to the left of box $i'$) on the $x$ axis.
* Box $i$ is located at a point $B_i$ of the $(x,y)$ plane and object $j$ is located at $O_j$.

* We want to find an arrangement of objects such that:
    * each box contains exactly one object,
    * each object is stored in one box,
    * the total distance from object $j$ to its storage box is minimal.

* First, we solve the problem described, and then we add two new constraints and examine how the cost (and solution) changes.
    * From the first solution, we impose that object #1 is assigned to the box immediately to the left of object #2.
    * Then we impose that object #5 is assigned to a box next to the box of object #6.

In [17]:
import docplex.mp
from math import sqrt
import random

In [18]:
# Setup Data
N = 3 #8
box_range = range(1, N+1)
obj_range = range(1, N+1)

o_xmax = N*10
o_ymax = 2*N
box_coords = {b: (10*b, 1) for b in box_range}

obj_coords= {1: (140, 6), 2: (146, 8), 3: (132, 14), 4: (53, 28), 
             5: (146, 4), 6: (137, 13), 7: (95, 12), 8: (68, 9), 9: (102, 18), 
             10: (116, 8), 11: (19, 29), 12: (89, 15), 13: (141, 4), 14: (29, 4), 15: (4, 28)}

# the distance matrix from box i to object j - we compute the square of distance to keep integer
distances = {}
for o in obj_range:
    for b in box_range:
        dx = obj_coords[o][0]-box_coords[b][0]
        dy = obj_coords[o][1]-box_coords[b][1]
        d2 = dx*dx + dy*dy
        distances[b, o] = d2

In [19]:
# Create the Model
from docplex.mp.model import Model
mdl = Model("boxes")

#### Define the decision variables

* For each box $i$ ($i$ in $1..N$) and object $j$ ($j$ in $1..N$), we define a binary variable $X_{i,j}$ equal to $1$ if and only if object $j$ is stored in box $i$.

In [20]:
# decision variables is a 2d-matrix
x = mdl.binary_var_matrix(box_range, obj_range, lambda ij: "x_%d_%d" %(ij[0], ij[1]))

#### Express the business constraints

* The sum of $X_{i,j}$ over both rows and columns must be equal to $1$, resulting in $2\times N$ constraints.

In [21]:
# one object per box
mdl.add_constraints(mdl.sum(x[i,j] for j in obj_range) == 1
                   for i in box_range)
    
# one box for each object
mdl.add_constraints(mdl.sum(x[i,j] for i in box_range) == 1
                  for j in obj_range)

mdl.print_information()

Model: boxes
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: none
 - problem type is: MILP


#### Express the objective

* The objective is to minimize the total distance between each object and its storage box.

In [22]:
# minimize total displacement
mdl.minimize( mdl.sum(distances[i,j] * x[i,j] for i in box_range for j in obj_range) )

In [23]:
mdl.export_as_lp(path='800-boxes.lp')

'800-boxes.lp'

#### Solve the model


In [24]:
mdl.print_information()

assert mdl.solve(), "!!! Solve of the model fails"

Model: boxes
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [25]:
mdl.report()
d1 = mdl.objective_value
#mdl.print_solution()

def make_solution_vector(x_vars):
    sol = [0]* N
    for i in box_range:
        for j in obj_range:
            if x[i,j].solution_value >= 0.5:
                sol[i-1] = j
                break
    return sol

def make_obj_box_dir(sol_vec):
    # sol_vec contains an array of objects in box order at slot b-1 we have obj(b)
    return { sol_vec[b]: b+1 for b in range(N)}
    
               
sol1 = make_solution_vector(x)
print("* solution: {0!s}".format(sol1))          

* model boxes solved with objective = 42983.000
* solution: [3, 1, 2]


#### Additional constraint #1

As an additional constraint, we want to impose that object #1 is stored immediately to the left of object #2.
As a consequence, object #2 cannot be stored in box #1, so we add:

In [11]:
mdl.add_constraint(x[1,2] == 0)

docplex.mp.LinearConstraint[](x_1_2,EQ,0)

Now, we must state that for $k \geq 2$ if $x[k,2] == 1$ then $x[k-1,1] == 1$; this is a logical implication that we express by a relational operator:

In [12]:
mdl.add_constraints(x[k-1,1] >= x[k,2]
                   for k in range(2,N+1))
mdl.print_information()

Model: boxes
 - number of variables: 225
   - binary=225, integer=0, continuous=0
 - number of constraints: 45
   - linear=45
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


Now let's solve again and check that our new constraint is satisfied, that is, object #1 is immediately left to object #2

In [13]:
ok2 = mdl.solve()
assert ok2, "solve failed"
mdl.report()
d2 = mdl.objective_value
sol2 = make_solution_vector(x)
print(" solution #2 ={0!s}".format(sol2))

* model boxes solved with objective = 8878.000
 solution #2 =[15, 11, 14, 4, 8, 12, 7, 9, 10, 3, 6, 13, 1, 2, 5]


The constraint is indeed satisfied, with a higher objective, as expected.

#### Additional constraint #2

Now, we want to add a second constraint to state that object #5 is stored in a box that is next to the box of object #6, either to the left or right.

In other words, when $x[k,6]$ is equal to $1$, then one of $x[k-1,5]$ and $x[k+1,5]$ is equal to $1$;
this is again a logical implication, with an OR in the right side.

We have to handle the case of extremities with care.

In [14]:
# forall k in 2..N-1 then we can use the sum on the right hand side
mdl.add_constraints(x[k,6] <= x[k-1,5] + x[k+1,5]
                  for k in range(2,N))
    
# if 6 is in box 1 then 5 must be in 2
mdl.add_constraint(x[1,6] <= x[2,5])

# if 6 is last, then 5 must be before last
mdl.add_constraint(x[N,6] <= x[N-1,5])

# we solve again
ok3 = mdl.solve()
assert ok3, "solve failed"
mdl.report()
d3 = mdl.objective_value

sol3 = make_solution_vector(x)
print(" solution #3 ={0!s}".format(sol3)) 

* model boxes solved with objective = 9078.000
 solution #3 =[15, 11, 14, 4, 8, 12, 7, 9, 10, 3, 13, 6, 5, 1, 2]


As expected, the constraint is satisfied; objects #5 and #6 are next to each other.
Predictably, the objective is higher.

### Step 5: Investigate the solution and then run an example analysis

Present the solution as a vector of object indices, sorted by box indices.
We use maptplotlib to display the assignment of objects to boxes.


In [15]:
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 12, 6

def display_solution(sol):
    obj_boxes = make_obj_box_dir(sol)
    xs = []
    ys = []
    for o in obj_range:
        b = obj_boxes[o]
        box_x = box_coords[b][0]
        box_y = box_coords[b][1]
        obj_x = obj_coords[o][0]
        obj_y = obj_coords[o][1]
        plt.text(obj_x, obj_y, str(o), bbox=dict(facecolor='red', alpha=0.5))
        plt.plot([obj_x, box_x], [obj_y, box_y])


ModuleNotFoundError: No module named 'matplotlib'

The first solution shows no segments crossing, which is to be expected.

In [ ]:
display_solution(sol1)

The second solution, by enforcing that object #1 must be to the left of object #2, introduces crossings.

In [ ]:
display_solution(sol2)

In [ ]:
display_solution(sol3)

In [ ]:

def display(myDict, title):
    if True: #env.has_matplotlib:
        N = len(myDict)
        labels = myDict.keys()
        values= myDict.values()

        try: # Python 2
            ind = xrange(N)  # the x locations for the groups
        except: # Python 3
            ind = range(N)
        width = 0.2      # the width of the bars

        fig, ax = plt.subplots()
        rects1 = ax.bar(ind, values, width, color='g')	
        ax.set_title(title)
        ax.set_xticks([ind[i]+width/2 for i in ind])
        ax.set_xticklabels( labels )	
        #ax.legend( (rects1[0]), (title) )

        plt.show()
    else:
        print("warning: no display")
        
from collections import OrderedDict
dists = OrderedDict()
dists["d1"]= d1 -8000
dists["d2"] = d2 - 8000
dists["d3"] = d3 - 8000
print(dists)

display(dists, "evolution of distance objective")

## Summary

You learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to formulate a Mathematical Programming model and solve it with CPLEX.

## References
* [CPLEX Modeling for Python documentation](http://ibmdecisionoptimization.github.io/docplex-doc/)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* Need help with DOcplex or to report a bug? Please go [here](https://stackoverflow.com/questions/tagged/docplex).
* Contact us at dofeedback@wwpdl.vnet.ibm.com.

Copyright &copy; 2017-2019 IBM. IPLA licensed Sample Materials.